# Experiment 003: Full Ensemble + Fractional Translation

Combine all available solutions (boc, saspav, smartmanoj) and apply fractional translation optimization.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import json
from tqdm import tqdm

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def load_solution(csv_path):
    """Load a solution CSV and return a dict of {n: [(x, y, deg), ...]}"""
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    solution = {}
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
        solution[n] = trees
    
    return solution

def score_config(trees_data):
    """Score a single N configuration given list of (x, y, deg) tuples"""
    tree_list = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    side = get_tree_list_side_length(tree_list)
    n = len(trees_data)
    return side ** 2 / n

print('Functions defined.')

In [ ]:
# Load all available solutions
print('Loading solutions...')

boc_path = '/home/code/exploration/preoptimized/submission.csv'
saspav_path = '/home/code/exploration/saspav/santa-2025.csv'
smartmanoj_path = 'smartmanoj_submission.csv'

solutions = {}

print('Loading bucket-of-chump...')
solutions['boc'] = load_solution(boc_path)
print(f'  Loaded {len(solutions["boc"])} N configurations')

print('Loading saspav...')
solutions['saspav'] = load_solution(saspav_path)
print(f'  Loaded {len(solutions["saspav"])} N configurations')

print('Loading smartmanoj...')
solutions['smartmanoj'] = load_solution(smartmanoj_path)
print(f'  Loaded {len(solutions["smartmanoj"])} N configurations')

print(f'\nTotal solutions loaded: {len(solutions)}')

In [ ]:
# Create ensemble by taking best per-N from all solutions
ensemble_solution = {}
source_wins = {name: 0 for name in solutions}

print('Creating ensemble from all solutions...')
for n in range(1, 201):
    best_score = float('inf')
    best_config = None
    best_source = None
    
    for name, sol in solutions.items():
        if n in sol:
            score = score_config(sol[n])
            if score < best_score:
                best_score = score
                best_config = sol[n]
                best_source = name
    
    if best_config is not None:
        ensemble_solution[n] = best_config
        source_wins[best_source] += 1
    
    if n <= 10 or n % 50 == 0:
        print(f'  N={n:3d}: best={best_source}, score={best_score:.6f}')

print(f'\nSource wins: {source_wins}')

# Calculate ensemble score
ensemble_total = sum(score_config(ensemble_solution[n]) for n in range(1, 201))
print(f'\nEnsemble total score: {ensemble_total:.6f}')
print(f'Target: 68.919154')
print(f'Gap to target: {ensemble_total - 68.919154:.6f}')

In [ ]:
# Implement fractional translation optimization
def has_overlap(trees_data, exclude_idx=None):
    """Check if any trees overlap (excluding one tree for movement testing)"""
    tree_list = [ChristmasTree(x, y, deg) for i, (x, y, deg) in enumerate(trees_data) if i != exclude_idx]
    if exclude_idx is not None:
        test_tree = ChristmasTree(*trees_data[exclude_idx])
        for t in tree_list:
            if test_tree.polygon.intersects(t.polygon) and not test_tree.polygon.touches(t.polygon):
                return True
    else:
        for i, t1 in enumerate(tree_list):
            for j, t2 in enumerate(tree_list):
                if i < j:
                    if t1.polygon.intersects(t2.polygon) and not t1.polygon.touches(t2.polygon):
                        return True
    return False

def fractional_translation(trees_data, max_iter=50):
    """Apply fractional translation to optimize bounding box"""
    # Convert to mutable list of floats
    config = [(float(x), float(y), float(deg)) for x, y, deg in trees_data]
    n = len(config)
    
    best_config = config.copy()
    best_side = get_tree_list_side_length([ChristmasTree(str(x), str(y), str(d)) for x, y, d in config])
    
    # Fractional steps and directions
    frac_steps = [0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001]
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    for iteration in range(max_iter):
        improved = False
        for i in range(n):
            for step in frac_steps:
                for dx, dy in directions:
                    # Try moving tree i
                    old_x, old_y, old_deg = best_config[i]
                    new_x = old_x + dx * step
                    new_y = old_y + dy * step
                    
                    # Create test config
                    test_config = best_config.copy()
                    test_config[i] = (new_x, new_y, old_deg)
                    
                    # Check for overlaps
                    tree_list = [ChristmasTree(str(x), str(y), str(d)) for x, y, d in test_config]
                    has_ovl = False
                    for j, t1 in enumerate(tree_list):
                        if has_ovl:
                            break
                        for k, t2 in enumerate(tree_list):
                            if j < k:
                                if t1.polygon.intersects(t2.polygon) and not t1.polygon.touches(t2.polygon):
                                    has_ovl = True
                                    break
                    
                    if not has_ovl:
                        new_side = get_tree_list_side_length(tree_list)
                        if new_side < best_side - 1e-12:
                            best_side = new_side
                            best_config = test_config
                            improved = True
        
        if not improved:
            break
    
    return [(str(x), str(y), str(d)) for x, y, d in best_config], best_side

print('Fractional translation function defined.')

In [ ]:
# Apply fractional translation to small N values (where it's tractable)
print('Applying fractional translation to small N values...')

optimized_solution = {n: list(ensemble_solution[n]) for n in ensemble_solution}
improvements = []

# Focus on N=2-30 where fractional translation is tractable
for n in tqdm(range(2, 31), desc='Optimizing'):
    old_score = score_config(optimized_solution[n])
    new_config, new_side = fractional_translation(optimized_solution[n], max_iter=20)
    new_score = new_side ** 2 / n
    
    if new_score < old_score - 1e-9:
        improvements.append((n, old_score, new_score, old_score - new_score))
        optimized_solution[n] = new_config

print(f'\nFound {len(improvements)} improvements:')
for n, old, new, delta in improvements:
    print(f'  N={n:3d}: {old:.6f} -> {new:.6f} (improvement: {delta:.9f})')

total_improvement = sum(delta for _, _, _, delta in improvements)
print(f'\nTotal improvement from fractional translation: {total_improvement:.9f}')

In [ ]:
# Calculate final score
final_total = sum(score_config(optimized_solution[n]) for n in range(1, 201))

print(f'Original ensemble score: {ensemble_total:.6f}')
print(f'After fractional translation: {final_total:.6f}')
print(f'Improvement: {ensemble_total - final_total:.9f}')
print(f'Target: 68.919154')
print(f'Gap to target: {final_total - 68.919154:.6f}')

In [ ]:
# Create submission CSV
rows = []
for n in range(1, 201):
    for i, (x, y, deg) in enumerate(optimized_solution[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{x}',
            'y': f's{y}',
            'deg': f's{deg}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('submission.csv', index=False)
print(f'Saved submission with {len(submission_df)} rows')

In [ ]:
# Save metrics
metrics = {
    'cv_score': final_total,
    'ensemble_score': ensemble_total,
    'improvement_from_frac_trans': ensemble_total - final_total,
    'source_wins': source_wins,
    'num_improvements': len(improvements),
    'target': 68.919154,
    'gap_to_target': final_total - 68.919154
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics: {metrics}')